In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"

import json
from math import log2, ceil
from tqdm import tqdm

In [2]:
puzzle_info = pd.read_csv("puzzle_info.csv")
puzzles = pd.read_csv("puzzles.csv")
mask_wildcards = puzzles.num_wildcards==0
puzzles = puzzles[mask_wildcards]
puzzles_list = list(puzzles.puzzle_type.unique())

In [3]:
puzzles.puzzle_type.value_counts()

cube_3/3/3       99
cube_4/4/4       50
cube_2/2/2       30
cube_5/5/5       27
wreath_6/6       15
wreath_7/7       13
globe_3/4        10
globe_1/8         9
globe_3/33        7
cube_6/6/6        7
wreath_12/12      7
wreath_21/21      5
cube_10/10/10     4
cube_7/7/7        4
globe_6/4         4
globe_6/8         4
cube_8/8/8        3
cube_19/19/19     3
cube_33/33/33     3
cube_9/9/9        3
globe_1/16        3
globe_2/6         3
wreath_33/33      2
globe_8/25        2
globe_6/10        1
Name: puzzle_type, dtype: int64

## Save generators

In [17]:
print(f"p   | {'puzzle':16s} | #actions | # santa states   | # unique elements |")
for j in range(puzzle_info.shape[0]):
    allowed_moves = eval(puzzle_info.iloc[j].allowed_moves)
    actions = [v for (k, v) in allowed_moves.items()]
    names = [k for (k, v) in allowed_moves.items()]

    actions += [list(np.argsort(a)) for a in actions]
    names += [name + "'" for name in names]

    actions = [[int(i) for i in p] for p in actions]

    generators = {'actions':actions, 'names':names}
        
    puzzle = puzzle_info.puzzle_type.iloc[j]
    solution_states = puzzles[puzzles.puzzle_type == puzzle].solution_state.unique()
    
    with open(f'../generators/p{j:03d}.json', 'w') as f:
        json.dump(generators, f)
    
    print(f"{j:03d} | {puzzle_info.iloc[j].puzzle_type:16s} | {len(actions):3d}      | {str(puzzles[puzzles.puzzle_type == puzzle].solution_state.value_counts().to_list()):16s} | {str([np.unique(s.split(';')).shape[0] for s in solution_states]):17s} |")

p   | puzzle           | #actions | # santa states   | # unique elements |
000 | cube_2/2/2       |  12      | [20, 5, 5]       | [6, 24, 6]        |
001 | cube_3/3/3       |  18      | [82, 10, 7]      | [6, 6, 54]        |
002 | cube_4/4/4       |  24      | [43, 4, 3]       | [6, 6, 96]        |
003 | cube_5/5/5       |  30      | [19, 5, 3]       | [6, 6, 150]       |
004 | cube_6/6/6       |  36      | [6, 1]           | [6, 6]            |
005 | cube_7/7/7       |  42      | [4]              | [6]               |
006 | cube_8/8/8       |  48      | [3]              | [6]               |
007 | cube_9/9/9       |  54      | [3]              | [6]               |
008 | cube_10/10/10    |  60      | [4]              | [6]               |
009 | cube_19/19/19    | 114      | [3]              | [6]               |
010 | cube_33/33/33    | 198      | [1, 1, 1]        | [6, 6, 6534]      |
011 | wreath_6/6       |   4      | [15]             | [3]               |
012 | wreath_7/7       | 

## Save datasets

In [5]:
def generate_inverse_moves(moves):
    """Generate the inverse moves for a given list of moves."""
    inverse_moves = [0] * len(moves)
    for i, move in enumerate(moves):
        if "'" in move:  # It's an a_j'
            inverse_moves[i] = moves.index(move.replace("'", ""))
        else:  # It's an a_j
            inverse_moves[i] = moves.index(move + "'")
    return inverse_moves

def random_step(states, last_moves):
    """Perform a random step while avoiding inverse moves."""
    possible_moves = torch.ones((states.size(0), all_moves.size(0)), dtype=torch.bool)
    possible_moves[torch.arange(states.size(0)), inverse_moves[last_moves]] = False
    next_moves = torch.multinomial(possible_moves.float(), 1).squeeze()
    new_states = torch.gather(states, 1, all_moves[next_moves])
    return new_states, next_moves

In [6]:
for j in range(puzzle_info.shape[0]):
    solution_states = puzzles[puzzles.puzzle_type == puzzle_info.puzzle_type.iloc[j]].solution_state.unique()
    puzzles_j = puzzles[puzzles.puzzle_type == puzzle_info.puzzle_type.iloc[j]]

    with open(f'../generators/p{j:03d}.json', 'r') as f:
        all_moves, move_names = json.load(f).values()
        all_moves = torch.tensor(all_moves, dtype=torch.int64)

    for k in range(len(solution_states)):
        solution_state = solution_states[k].split(";")
        e2n = {e:j for (j, e) in enumerate(np.unique(solution_state))}
        solution_state = [e2n[e] for e in solution_state]
        initial_states = puzzles_j[puzzles_j.solution_state == solution_states[k]].initial_state.to_list()
        initial_states = [[e2n[e] for e in state.split(";")] for state in initial_states]

        # 8, 16, 32, 64
        # 3, 4, 5, 6
        num_elements = max(e2n.values())
        b = max(3, ceil(log2(log2(num_elements))))
        dtype = {3:torch.int8, 4:torch.int16, 5:torch.int32, 6:torch.int64}[b]

        if num_elements >= 2**(2**b-1):
            shift = 2**(2**b-1)
        else:
            shift = 0

        solution_state = torch.tensor(np.array(solution_state) - shift, dtype=dtype)
        initial_states = torch.tensor(np.array(initial_states) - shift, dtype=dtype)

        inverse_moves = torch.tensor(generate_inverse_moves(move_names), dtype=torch.int64)

        N = 100
        last_moves = torch.full((N,), -1, dtype=torch.int64)
        rnd_states = solution_state[None].expand(N, -1)

        for _ in range(10_000):
            rnd_states, last_moves = random_step(rnd_states, last_moves)

        mask = torch.randint(0, 2, (N,), dtype=torch.bool)
        rnd_states[mask], _ = random_step(rnd_states[mask], last_moves[mask])

        torch.save(rnd_states, f"../datasets/p{j:03d}-t{k:03d}-rnd.pt")
        torch.save(initial_states, f"../datasets/p{j:03d}-t{k:03d}-santa.pt")
        torch.save(solution_state, f"../targets/p{j:03d}-t{k:03d}.pt")
        
        print(f"{j:03d}.{k:03d}", end='\r')